In [1]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import datetime

In [2]:
def animate(x, mesh, sleep = 0.1, periods=1, stepsperPeriod=50, animate_domains = None, *args, **kwargs):
    import time
    import numpy as np

    ti = np.linspace(0, np.pi*2*periods, stepsperPeriod*periods)
    t_par = Parameter(0)

    if animate_domains != None:
        animate_CF = mesh.MaterialCF({dom[0] :exp(1j*t_par*dom[1]) for dom in animate_domains}, default=1)
    else:
        animate_CF = exp(1j*t_par)

    scene = Draw(x.real + x.imag*animate_CF.real, mesh, *args, **kwargs)
    for t in ti:
        t_par.Set(t)
        scene.Redraw()
        time.sleep(sleep)

In [3]:
print(datetime.datetime.today().day)

13


In [7]:
wp = WorkPlane()


triag = lambda width, height, shift : wp.MoveTo(0, shift).LineTo(width/2, shift).LineTo(0, shift+height).LineTo(-width/2, shift).Close().Face()

trunc_height = 1
trunc_width = 0.2

# trunk
tree = wp.MoveTo(-trunc_width/2, 0).Rectangle(trunc_width, trunc_height).Face()
tree.name="trunk"


widths = [3, 2.5, 1.9, 1.3, 1]
heights = [1, 0.8, 0.7, 0.6, 0.5]
shifts = [trunc_height+ s for s in [0.0, 0.5, .9, 1.4, 1.8]]


# branches
tops = [triag(w, h, s) for w, h, s in zip(widths, heights, shifts)]
def rename(t, n): 
    t.name = n
[rename(t, "top") for t in tops]

# balls
ball = lambda x, y, r : wp.MoveTo(x, y).Circle(r).Face()
xyrc_balls = [
                (0.1, 2, 0.05, 0.1), 
                (-0.9, 1.2, 0.07, 0.3 + 0.1j), 
                (0, 3, 0.05, 0.4),
                (-.2, 1.5, 0.06, 0.4 + 0.2j), 
                (.1, 2.5, 0.07, 0.8 + 0.2j),
                (-.7, 1.7, 0.09, 0.5 + 0.5j),
                (0.8, 1.2, 0.09, 0.3 + 0.2j)]
balls = [ball(x, y, r) for x, y, r, _ in xyrc_balls]
[rename(balls[i], f"ball{i}") for i in range(len(balls))]


  
# candles
width_candle = 0.05
height_candle = 0.2
color_candle = 1

candle = lambda x, y : wp.MoveTo(x, y).Rectangle(width_candle, height_candle).Face()
xyc_candles = [ (1.1, 1.55, color_candle), 
                (1.4, 1.05, color_candle), 
                (0.5, 1.4, color_candle), 
                (0.7, 1.95, color_candle), 
                (-.3, 2, color_candle), 
                (-.5, 1.34, color_candle), 
                (0.1, 3, color_candle), 
                (-.1, 1.1 , color_candle), 
                (-1.1, 1.1 , color_candle), 
                (-1.4, 1.05 , color_candle), 
                (-1.2, 1.15 , color_candle), 
                (-1.15, 1.55 , color_candle), 
                (0.3, 1.55 , color_candle), 
                (0.35, 1.05 , color_candle), 
                (-0.45, 1.75 , color_candle), 
                (-0.55, 2.1 , color_candle), 
                (-0.5, 2.5 , color_candle), 
                (-0.1, 2.4 , color_candle), 
                (0.4, 2.5 , color_candle), 
                (-0.2, 3. , color_candle), 
                (-0.8, 2. , color_candle), 
                (-0, 1.6 , color_candle), 
                (-0.7, 1.1 , color_candle), 
                (0-width_candle/2, 3.25 , color_candle), 
                ]
print(len(xyc_candles))
candles = [candle(x, y) for x, y, _ in xyc_candles]
[rename(candles[i], f"candle{i}") for i in range(len(candles))]

# flames
today = datetime.datetime.today()
flame_diameter = width_candle
xyc_flames = xyc_candles.copy()
if today.month == 12:
    xyc_flames = xyc_flames[:today.day]
    flames = [ball(x+width_candle/2, y + height_candle+width_candle/2, flame_diameter) for x, y, _ in xyc_flames]
    [rename(flames[i], f"flame{i}") for i in range(len(flames))]
    mesh = Mesh(OCCGeometry(Glue([tree, Fuse(tops)- Fuse(balls) - Fuse(candles) - Fuse(flames), Fuse(balls), Fuse(candles), Fuse(flames)] ), dim = 2).GenerateMesh(maxh=5))

else:
    mesh = Mesh(OCCGeometry(Glue([tree, Fuse(tops)- Fuse(balls) - Fuse(candles) , Fuse(balls), Fuse(candles)] ), dim = 2).GenerateMesh(maxh=5))






settings = {"Objects":{"Wireframe":False, "Edges":False}, "Colormap":{"autoscale":False, "min": 0.1, "max":1, "ncolors":100}}
colors = {"top":0.6, "trunk":0.86}
for i in range(len(balls)):
    colors.update({f"ball{i}":xyrc_balls[i][3]})
for i in range(len(candles)):
    colors.update({f"candle{i}":xyc_candles[i][2]})



if today.month == 12:
    for i in range(len(flames)):
        colors.update({f"flame{i}":0.85 + 1j*0.05 })

# animate(mesh.MaterialCF(colors, default=0), mesh, sleep=0.1, min = 0,max = 1, settings=settings, animate_domains=["ball.*"], stepsperPeriod=20, periods=5)
animate(mesh.MaterialCF(colors, default=0), mesh, sleep=0.1, min = 0,max = 1, settings=settings, animate_domains=[("flame.*", 3), ("ball.*", 0.7)], stepsperPeriod=20, periods=5)


24


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…